In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict


In [11]:
from core.Data_fetcher import DataFetcher
from core.IV_simplifier import IVSimplifier
from core.ValuationCalculator import ValuationCalculator

In [41]:
import pandas as pd
from typing import List, Dict, Optional
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import time
from collections import defaultdict
from core.Data_fetcher import DataFetcher
from core.IV_simplifier import IVSimplifier


# === CONFIGURATION ===
# Map Wikipedia GICS sectors to standardized names
SECTOR_MAPPING = {
    "Information Technology": "Technology",
    "Health Care": "Healthcare",
    "Financials": "Financials", 
    "Communication Services": "Communication Services",
    "Consumer Discretionary": "Consumer Discretionary",
    "Energy": "Energy",
    "Utilities": "Utilities",
    "Consumer Staples": "Consumer Staples",
    "Real Estate": "Real Estate",
    "Industrials": "Industrials", 
    "Materials": "Materials"
}

print("="*60)
print("S&P 500 SECTOR ANALYSIS")
print("="*60)

# === 1. WEB SCRAPING ===
print("\n1. Scraping S&P 500 data from Wikipedia...")
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", {"id": "constituents"})
    rows = table.find_all("tr")
    
    # === 2. DATA EXTRACTION ===
    data = []
    for row in rows[1:]:
        cols = row.find_all("td")
        if len(cols) >= 8:
            symbol = cols[0].text.strip().replace(".", "-")
            security = cols[1].text.strip()
            sector = cols[2].text.strip()
            
            data.append({
                "Symbol": symbol,
                "Security": security,
                "GICS Sector": sector,
                "GICS Sub-Industry": cols[3].text.strip(),
                "Headquarters": cols[4].text.strip(),
                "Date Added": cols[5].text.strip(),
                "CIK": cols[6].text.strip(),
                "Founded": cols[7].text.strip()
            })
    
    sp500_df = pd.DataFrame(data)
    print(f"✓ Scraped {len(sp500_df)} S&P 500 companies")
    
except Exception as e:
    print(f"✗ Failed to scrape S&P 500 data: {e}")
    raise

# === 3. SECTOR ORGANIZATION ===
sector_tickers = sp500_df.groupby("GICS Sector")["Symbol"].apply(list).to_dict()

# Create symbol-to-sector mapping from Wikipedia
wiki_symbol_to_sector = {}
for _, row in sp500_df.iterrows():
    wiki_sector = row["GICS Sector"]
    mapped_sector = SECTOR_MAPPING.get(wiki_sector, wiki_sector)
    wiki_symbol_to_sector[row["Symbol"]] = mapped_sector

print(f"\nSectors found: {len(sector_tickers)}")
for sector, tickers in sector_tickers.items():
    print(f"  {sector}: {len(tickers)} companies")

# === 4. DATA FETCHING ===
print("\n2. Fetching financial data...")

# For testing, you can limit to a subset
# TESTING: Uncomment the next line to test with fewer companies
sector_tickers = {k: v[:5] for k, v in list(sector_tickers.items())[:3]}

all_tickers = []
for sector in sector_tickers.keys():
    all_tickers.extend(sector_tickers[sector])

print(f"Total tickers to fetch: {len(all_tickers)}")

try:
    aggregator = MultiCompanyAggregator(DataFetcher, IVSimplifier)
    aggregator.fetch_multiple_companies(all_tickers, parallel=False, max_workers=10)
    time.sleep(0.5)
    
    print(f"✓ Successfully fetched {len(aggregator.company_data)} companies")
    
except Exception as e:
    print(f"✗ Error during data fetching: {e}")
    raise

# === 5. CREATE COMBINED DATAFRAME ===
print("\n3. Creating combined dataframe...")

# Don't include 'Date' in metrics - it's already handled
metrics = ['P/E Ratio', 'Basic EPS', 'Free Cash Flow', 
           'Operating Cash Flow', 'Investing Cash Flow', 'Annual Dividends', 
           'Net Income', 'Diluted EPS', 'Share Price', 'Total Assets', 
           'Total Liabilities Net Minority Interest',
           'Total Equity Gross Minority Interest', 'Basic Average Shares', 
           'Diluted Average Shares']

try:
    combined_df = aggregator.create_combined_dataframe(
        metrics=metrics, 
        latest_year_only=False
    )
    
    # Override Sector column with Wikipedia mapping (yfinance sectors may differ)
    combined_df['Sector'] = combined_df['Ticker'].map(wiki_symbol_to_sector)
    
    # Remove rows where sector mapping failed
    combined_df = combined_df.dropna(subset=['Sector'])
    
    print(f"✓ Combined dataframe created: {combined_df.shape}")
    
except Exception as e:
    print(f"✗ Error creating combined dataframe: {e}")
    raise

# === 6. DATA CLEANING FUNCTION ===
def clean_sector_data(df):
    """Clean and process data for a sector"""
    if df is None or df.empty:
        return None
    
    df_clean = df.copy()
    
    # Ensure Year column exists and is numeric
    if 'Year' not in df_clean.columns:
        warnings.warn("Year column not found in dataframe")
        return None
    
    df_clean['Year'] = pd.to_numeric(df_clean['Year'], errors='coerce')
    
    # Filter valid years (reasonable range)
    df_clean = df_clean[df_clean['Year'].between(2000, 2030)]
    
    # Convert numeric columns
    numeric_cols = ['Market Cap', 'Diluted EPS', 'P/E Ratio', 'Free Cash Flow']
    for col in numeric_cols:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # Remove rows with missing essential data
    df_clean = df_clean.dropna(subset=['Year', 'Market Cap', 'Diluted EPS'])
    
    # Remove rows with zero or negative market cap
    df_clean = df_clean[df_clean['Market Cap'] > 0]
    
    return df_clean

# === 7. PROCESS ALL SECTORS ===
print("\n4. Processing sector data...")

sector_data = {}
unique_sectors = combined_df['Sector'].unique()

for sector_name in unique_sectors:
    sector_df = combined_df[combined_df['Sector'] == sector_name].copy()
    
    if not sector_df.empty:
        cleaned_df = clean_sector_data(sector_df)
        if cleaned_df is not None and not cleaned_df.empty:
            sector_data[sector_name] = cleaned_df
            print(f"  ✓ {sector_name}: {len(cleaned_df)} rows, {len(cleaned_df['Ticker'].unique())} companies")
        else:
            print(f"  ✗ {sector_name}: No valid data after cleaning")
    else:
        print(f"  ✗ {sector_name}: No data found")

# === 8. SEQUENTIAL YEAR FUNCTION (Fixed) ===
def convert_to_sequential_years(df):
    """
    Convert absolute years to sequential years for the entire sector.
    Sequential Year 1 = earliest year with data across all companies in sector.
    """
    if df is None or df.empty:
        return None
    
    df_seq = df.copy()
    
    # Get the minimum year across all companies in this sector
    min_year = df_seq['Year'].min()
    
    # Calculate sequential year relative to sector's earliest year
    df_seq['Sequential_Year'] = df_seq['Year'] - min_year + 1
    
    return df_seq

# === 9. CALCULATE WEIGHTED EPS ===
def calculate_weighted_eps(df):
    """
    Calculate market-cap weighted EPS for each sequential year.
    """
    if df is None or df.empty:
        return None
    
    df_seq = convert_to_sequential_years(df)
    
    # Group by sequential year and calculate weighted average
    weighted_eps = df_seq.groupby('Sequential_Year').apply(
        lambda x: (x['Diluted EPS'] * x['Market Cap']).sum() / x['Market Cap'].sum()
        if x['Market Cap'].sum() > 0 else None
    ).dropna().sort_index()
    
    return weighted_eps

# === 10. CALCULATE FOR ALL SECTORS ===
print("\n5. Calculating weighted EPS by sector...")

sector_eps = {}
for sector_name, sector_df in sector_data.items():
    eps_data = calculate_weighted_eps(sector_df)
    if eps_data is not None and len(eps_data) > 0:
        sector_eps[sector_name] = eps_data
        print(f"  ✓ {sector_name}: {len(eps_data)} data points")

# === 11. PLOTTING ===
print("\n6. Creating visualization...")

plt.figure(figsize=(16, 9))

# Enhanced color palette
sector_styles = {
    'Technology': ('#1f77b4', 'o'),
    'Healthcare': ('#2ca02c', 's'), 
    'Financials': ('#d62728', '^'),
    'Communication Services': ('#9467bd', 'D'),
    'Energy': ('#ff7f0e', 'v'),
    'Consumer Discretionary': ('#8c564b', '<'),
    'Consumer Staples': ('#e377c2', '>'),
    'Industrials': ('#7f7f7f', 'p'),
    'Materials': ('#bcbd22', '*'),
    'Utilities': ('#17becf', 'h'),
    'Real Estate': ('#aec7e8', 'X')
}

plotted_count = 0
for sector_name, eps_data in sector_eps.items():
    style = sector_styles.get(sector_name, ('#000000', 'o'))
    color, marker = style
    
    if len(eps_data) > 0:
        plt.plot(eps_data.index, eps_data.values, 
                 marker=marker, linewidth=2.5, markersize=7,
                 color=color, label=sector_name, alpha=0.8)
        plotted_count += 1

plt.title('S&P 500: Market-Cap Weighted Diluted EPS by Sector (Sequential Years)', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Sequential Year (1 = Earliest Year with Data)', fontsize=13)
plt.ylabel('Weighted Diluted EPS ($)', fontsize=13)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()

print(f"✓ Plotted {plotted_count} sectors")
print("\n" + "="*60)
print("Analysis complete!")
print("="*60)

plt.show()

# === 12. OPTIONAL: SAVE RESULTS ===
# Uncomment to save results
combined_df.to_csv('sp500_combined_data.csv', index=False)
print("\n✓ Data saved to sp500_combined_data.csv")

S&P 500 SECTOR ANALYSIS

1. Scraping S&P 500 data from Wikipedia...
✓ Scraped 503 S&P 500 companies

Sectors found: 11
  Communication Services: 24 companies
  Consumer Discretionary: 50 companies
  Consumer Staples: 37 companies
  Energy: 22 companies
  Financials: 75 companies
  Health Care: 60 companies
  Industrials: 79 companies
  Information Technology: 68 companies
  Materials: 26 companies
  Real Estate: 31 companies
  Utilities: 31 companies

2. Fetching financial data...
Total tickers to fetch: 15
Fetching GOOGL...
Fetching GOOG...
Fetching T...
Info: Removed 1 column(s) with missing data
Fetching CHTR...
Info: Removed 1 column(s) with missing data
Fetching CMCSA...
Info: Removed 1 column(s) with missing data
Fetching ABNB...
Info: Removed 1 column(s) with missing data
Fetching AMZN...
Info: Removed 1 column(s) with missing data
Fetching APTV...
Info: Removed 1 column(s) with missing data
Fetching AZO...
Info: Removed 2 column(s) with missing data
Fetching BBY...
Fetching MO.

c:\Users\joao_\Desktop\Data Science\Finance\market-mind\core\MultiCompanyAggregator.py:125: UserWarning: Error transforming data for GOOGL: cannot assemble with duplicate keys
  warnings.warn(f"Error transforming data for {ticker}: {e}")
c:\Users\joao_\Desktop\Data Science\Finance\market-mind\core\MultiCompanyAggregator.py:125: UserWarning: Error transforming data for GOOG: cannot assemble with duplicate keys
  warnings.warn(f"Error transforming data for {ticker}: {e}")
c:\Users\joao_\Desktop\Data Science\Finance\market-mind\core\MultiCompanyAggregator.py:125: UserWarning: Error transforming data for T: cannot assemble with duplicate keys
  warnings.warn(f"Error transforming data for {ticker}: {e}")
c:\Users\joao_\Desktop\Data Science\Finance\market-mind\core\MultiCompanyAggregator.py:125: UserWarning: Error transforming data for CHTR: cannot assemble with duplicate keys
  warnings.warn(f"Error transforming data for {ticker}: {e}")
c:\Users\joao_\Desktop\Data Science\Finance\market-min

ValueError: No valid data to combine after transformation.

In [46]:
df = aggregator.company_data['AAPL']  # or any other ticker
df.head()



,Year 4,Year 3,Year 2,Year 1
Date,2024,2023,2022,2021
Ticker,AAPL,AAPL,AAPL,AAPL
Market Cap,3899609055232,3899609055232,3899609055232,3899609055232
P/E Ratio,37.957551,27.52423,22.122334,24.431192
Basic EPS,6.11,6.16,6.15,5.67


In [47]:
aggregator.create_combined_dataframe(metrics=metrics, latest_year_only=False)
aggregator.combined_df.head()


c:\Users\joao_\Desktop\Data Science\Finance\market-mind\core\MultiCompanyAggregator.py:125: UserWarning: Error transforming data for GOOGL: cannot assemble with duplicate keys
  warnings.warn(f"Error transforming data for {ticker}: {e}")
c:\Users\joao_\Desktop\Data Science\Finance\market-mind\core\MultiCompanyAggregator.py:125: UserWarning: Error transforming data for GOOG: cannot assemble with duplicate keys
  warnings.warn(f"Error transforming data for {ticker}: {e}")
c:\Users\joao_\Desktop\Data Science\Finance\market-mind\core\MultiCompanyAggregator.py:125: UserWarning: Error transforming data for T: cannot assemble with duplicate keys
  warnings.warn(f"Error transforming data for {ticker}: {e}")
c:\Users\joao_\Desktop\Data Science\Finance\market-mind\core\MultiCompanyAggregator.py:125: UserWarning: Error transforming data for CHTR: cannot assemble with duplicate keys
  warnings.warn(f"Error transforming data for {ticker}: {e}")
c:\Users\joao_\Desktop\Data Science\Finance\market-min

ValueError: No valid data to combine after transformation.

In [49]:
aggregator.company_data

{'GOOGL':                                                  Year 4          Year 3  \
 Date                                               2024            2023   
 Ticker                                            GOOGL           GOOGL   
 Market Cap                                3066071089152   3066071089152   
 P/E Ratio                                      23.20847       23.755674   
 Basic EPS                                          8.13            5.84   
 Free Cash Flow                            72764000000.0   69495000000.0   
 Operating Cash Flow                      125299000000.0  101746000000.0   
 Investing Cash Flow                      -45536000000.0  -27063000000.0   
 Annual Dividends                                    0.6             0.0   
 Net Income                               100118000000.0   73795000000.0   
 Diluted EPS                                        8.04             5.8   
 Share Price                                   188.68486      138.733139   
 To

In [52]:
df = aggregator.company_data['GOOGL']  # pick a problematic ticker

# Show duplicate row indices
duplicate_rows = df.index[df.index.duplicated()]
print("Duplicate row indices:")
print(duplicate_rows)

# Show the actual duplicate rows
if not duplicate_rows.empty:
    print("\nDuplicate rows data:")
    print(df.loc[duplicate_rows])


Duplicate row indices:
Index([], dtype='object')


In [53]:
# Show duplicate column names
duplicate_columns = df.columns[df.columns.duplicated()]
print("Duplicate columns:")
print(duplicate_columns)

# Show the actual duplicate columns
if not duplicate_columns.empty:
    print("\nDuplicate columns data:")
    print(df[duplicate_columns])


Duplicate columns:
Index([], dtype='object')


In [54]:
for ticker, df in aggregator.company_data.items():
    dup_rows = df.index[df.index.duplicated()]
    dup_cols = df.columns[df.columns.duplicated()]
    
    if len(dup_rows) > 0 or len(dup_cols) > 0:
        print(f"\nTicker: {ticker}")
        if len(dup_rows) > 0:
            print(f"  Duplicate rows: {dup_rows.tolist()}")
        if len(dup_cols) > 0:
            print(f"  Duplicate columns: {dup_cols.tolist()}")


In [55]:
df = df[~df.index.duplicated(keep='first')]
df = df.loc[:, ~df.columns.duplicated(keep='first')]


In [56]:
df


,Year 5,Year 4,Year 3,Year 2
Date,2025,2024,2023,2022
Ticker,CPB,CPB,CPB,CPB
Market Cap,9249633280,9249633280,9249633280,9249633280
P/E Ratio,15.609333,23.42237,14.652892,17.52819
Basic EPS,2.02,1.9,2.87,2.51
Free Cash Flow,705000000.0,668000000.0,773000000.0,939000000.0
Operating Cash Flow,1131000000.0,1185000000.0,1143000000.0,1181000000.0
Investing Cash Flow,-187000000.0,-3128000000.0,-340000000.0,-230000000.0
Annual Dividends,1.54,1.48,1.48,1.48
Net Income,602000000.0,567000000.0,858000000.0,757000000.0
